In [0]:
# DESCRIPTION: Joins Bronze tables to create a comprehensive Silver 'f1_results' table.
# ---------------------------------------------------------

from pyspark.sql.functions import col, current_timestamp

# 1. Read Silver Dimensions & Bronze Facts
# ---------------------------------------------------------
drivers_df = spark.read.format("delta").load("abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/drivers")
races_df = spark.read.format("delta").load("abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/races")
circuits_df = spark.read.format("delta").load("abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/circuits")
constructors_df = spark.read.format("delta").load("abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/constructors")

results_df = spark.read.json("abfss://bronze@YOUR_ACCOUNT.dfs.core.windows.net/results")

# 2. Rename & Clean Results
# ---------------------------------------------------------
results_clean = results_df \
    .withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("positionOrder", "position") \
    .withColumnRenamed("points", "actual_points") \
    .drop("url")

# 3. Join Everything Together (The "Silver" Magic)
f1_results_df = results_clean.join(races_df, results_clean.race_id == races_df.race_id, "inner") \
                             .join(drivers_df, results_clean.driver_id == drivers_df.driver_id, "inner") \
                             .join(constructors_df, results_clean.constructor_id == constructors_df.constructor_id, "inner") \
                             .join(circuits_df, races_df.circuit_id == circuits_df.circuit_id, "inner") \
                             .select(
                                 races_df.year.alias("season"),
                                 races_df.name.alias("race_name"),
                                 races_df.date.alias("race_date"),
                                 circuits_df.location.alias("circuit_location"),
                                 drivers_df.forename.alias("driver_first_name"),
                                 drivers_df.surname.alias("driver_last_name"),
                                 drivers_df.nationality.alias("driver_nationality"),
                                 constructors_df.name.alias("team_name"),
                                 results_clean.position,
                                 results_clean.actual_points
                             )

# 4. Write to Silver
# ---------------------------------------------------------
silver_path = "abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/f1_results"
f1_results_df.write.mode("overwrite").partitionBy("season").format("delta").save(silver_path)

print("✅ transform_race_results completed! Silver table 'f1_results' created.")